In [1]:
from pyelasticsearch.client import ElasticSearch
import json
import requests
import pandas as pd
import math
import numpy as np
import boto3
import re
import collections
from collections import defaultdict
from dateutil import parser
from datetime import datetime
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz
import csv
import re

In [2]:
config = json.load(open('../config/configsql.json', 'r'))
aws_access_key_id=config['dev']['aws']['key']
aws_secret_access_key=config['dev']['aws']['secret']
region = 'us-east-1'
endpoint = config['dev']['endpoint']
bucketname=config['dev']['bucketname']['bucketname']
s3r = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
s3c = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key)
index_name1 = 'candidates_cv'
index_name2 = 'parsed_resume'
es = ElasticSearch(endpoint, port=443)

In [32]:
d_all = es.search(index = index_name2, query={
                                     "query": {"match_all": {}},
                                     "size":  4
                                     })

In [26]:
def run_test(es, validation_csv_filename):
    tdf = pd.read_csv(validation_csv_filename).head(50) #    tdf = tdf.head(100)
    tdf.columns = [i.lower() for i in tdf.columns]
    if tdf.shape[0] == 0:
        return 0.0
    tdf.sort_values('id', inplace=True)
    failures = []
    success_count = 0
    for r in range(tdf.shape[0]):
        row = tdf.iloc[r].to_dict()
        id = row['id']
        print(id)
        user_record = es.search(index=index_name2,query={"query" : {"match":{"_id": id}}})
        if len(user_record['hits']['hits']) > 0:
            user = user_record['hits']['hits'][0]
            is_valid = validation_check(user,row)
            if is_valid:
                success_count += 1
            else:
                failures.append({"correct": row, "user_record": get_educations(user)})
    return success_count / tdf.shape[0], failures

In [ ]:
def validation_check(user,row):
    educations = get_educations(user)
    match_any = False
    for edu in educations:
        success = True
        for value in ["institution"]:#, "degree_type", "years_low", "years_high"]: 
            resume_value = str(edu[value]).lower()
            validation_value = str(row[value]).lower()
            print(resume_value+'----'+validation_value)
            print(fuzz.ratio(resume_value,validation_value))
            fuzz_score = fuzz.ratio(resume_value,validation_value)
            if fuzz_score < 70:
                success = False
        if success:
            match_any = True
        if success:
            return True
        else:
            return False

In [27]:
success, failures = run_test(es, 'Validations.csv')
print(success)

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-04text/resumes_career_page1_2018-04_adityarputra@gmail.com_ECV2_Aditya-Rachman-Putra_EN.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-04text/resumes_career_page1_2018-04_brenthauer@gmail.com_Brent Hauer Resume 2017.pdf.txt
lawrence university----belmont university – nashville, tn
49
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-04text/resumes_career_page1_2018-04_brenthauer@gmail.com_Brent Hauer Resume 2017.pdf.txt
lawrence university----lawrence university – appleton, wi
72
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-04text/resumes_career_page1_2018-04_bryanwnonni@gmail.com_Bryan Nonni March 18.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-04text/resumes_career_page1_2018-04_bryanwnonni@gmail.com_Bryan Nonni March 18.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resume

In [74]:
get_educations(p_s)

None
{'institution': 'University of Pune', 'degree_type': 'BS', 'years_low': '2008', 'years_high': '2015'}
None
None
None
None
None
None
None
None
None
None
None
None
None
{'institution': 'University of Washington', 'degree_type': 'BS', 'years_low': '2008', 'years_high': '2015'}
None
None
None
None
None
None
None
None
None
None


[{'degree_type': 'BS',
  'institution': 'University of Pune',
  'years_high': '2015',
  'years_low': '2008'},
 {'degree_type': 'BS',
  'institution': 'University of Washington',
  'years_high': '2015',
  'years_low': '2008'}]

In [71]:
def get_educations(parsed_resume):
    educations = []
    education = parsed_resume['_source']['education']
    id = parsed_resume['_id']
    user_record = es.search(index=index_name1,query={"query" : {"match":{"_id": id}}})
    user_record_degree = user_record['hits']['hits'][0]['_source']
    if len(parsed_resume['_source']['education']['entities']) > 0:
        text = [i['Text'] for i in parsed_resume['_source']['education']['entities']['Entities']]
        type = [i['Type'] for i in parsed_resume['_source']['education']['entities']['Entities']]
        start_pos = [i['BeginOffset'] for i in parsed_resume['_source']['education']['entities']['Entities']]
        pos_d = dict(zip(text, start_pos))
        my_dict = defaultdict(list)
        for k, v in zip(type, text):
            my_dict[k].append(v)
        organizations = list(set(my_dict['ORGANIZATION']))
        for organization in organizations:
            school_details = get_school_details(organization, pos_d, user_record_degree, my_dict)
            print(school_details)
            if school_details is not None:
                educations.append(school_details)
    return  educations

In [7]:
def parse_date(s):
    result = ""
    for ch in s:
        if ch.isdigit():
            result += ch
    return result

In [73]:
def get_school_details(orgname, pos_d, user_record_degree, my_dict):
    org_pos = pos_d[orgname]
    if orgname.lower().find('university') != -1:
        years = my_dict['DATE']
        years = [parse_date(i) for i in years]
        years_low = -1
        years_high = -1
        if len(years) > 0:
            years_low = min(years)
            years_high = max(years)
        degree_type = find_degree_type(org_pos,user_record_degree)
        edu_info = {"institution": orgname,"degree_type":degree_type, "years_low":years_low,"years_high":years_high}
        return edu_info
    else: 
        return None

In [79]:
def find_degree_type(univ_pos,user_record):
    values = {"bachelor":'BS',"master":'MS','phd':'phD','B.S.':'BS'}
    for k in user_record:
        if 'education' in k:
            degree_d = {}
            for key in values.keys():
                degree_pos = user_record['education'].lower().find(key)
                degree_d[degree_pos] = key
            degree_type = degree_d.get(univ_pos, degree_d[min(degree_d.keys(), key=lambda k: abs(k-univ_pos))])
            abbr_degree = values[degree_type]
    return abbr_degree

In [94]:
user_record_degree['education'].lower()

"university of washington, information school, seattle, wa sep 2014 may 2016\nmaster of science in information management (focus: data science & analytics) (gpa: 3.76) jul 2008 jun 2012\nuniversity of pune, india\nbachelors of engineering in electronics (grade first class with distinction)\nemployment\ndata solutions analytics intern at the walt disney company, seattle, wa aug 2015 ongoing\nworking with youtube api to programmatically collect disney's youtube channel owners' data, and perform marketing analysis &\nvisualization, to facilitate business decisions by higher management.\ndeveloping code in java and mysql to automate sweepstakes for espn, abc, disney on ice, honda, dior etc.\nsoftware developer intern (big data) at bank of america, charlotte, nc jun 2015 aug 2015\nworking on agile with systems engineering team understanding the hadoop stack, and developing logging framework for hive and pig\nhands on experience on hdfs, hadoop map reduce framework, pig macro, java udf, hive

In [99]:
user_record_degree['education'].lower().find('master')

76

In [113]:
i = user_record_degree['education'].lower().find('bachelors')
j = user_record_degree['education'].lower().find('in ', i)
k = user_record_degree['education'].lower().find('\n', j)
user_record_degree['education'].lower()[j:k]

'in electronics (grade first class with distinction)'

In [330]:
def find_major_type(user_record_degree):
    values = {"bachelor":'BS',"master":'MS','ph.d':'phD','b.s.':'BS','m.s':'MS'}
    for k in user_record_degree:
        if 'education' in k:
            degree_d = {}
            for key in values.keys():
                degree_pos = user_record_degree['education'].lower().find(key) 
                degree_d[key] = degree_pos 
                degree_d = {k: v for k, v in degree_d.items() if v >= 0}
            for key,value in degree_d.items():
                j = user_record_degree['education'].lower().find('in ', value)
                k = user_record_degree['education'].lower().find('\n', j)
                major = user_record_degree['education'].lower()[j+3:k]
                degree_d[key] = major
            return degree_d

In [331]:
find_major_type(user_record_degree)

{'b.s.': 'chemical physics, university of science and technology of china sept 2006 - jun 2010',
 'ph.d': 'physical chemistry, brown university sept 2010 - jan 2016'}

In [327]:
user_record_degree['education'].lower().find('b.s.')

134

In [310]:
user_record = es.search(index=index_name1,query={"query" : {"match":{"_id": 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Yan.Gao.Resume.pdf.txt'}}})

In [311]:
user_record_degree = user_record['hits']['hits'][0]['_source']

In [314]:
user_record_degree

{'awards': 'Published six papers on reputable journals and gave presentations on one national conference.\nBrown University: Graduate Travel Fellowship (2013), Graduate Fellowship (2011).\nUSTC: Outstanding student fellowship, three times (2006 - 2009).',
 'education': "Ph.D. in Physical Chemistry, Brown University Sept 2010 - Jan 2016\nSelected coursework: Computation Chemistry, Statistical Mechanics.\nB.S. in Chemical Physics, University of Science and Technology of China Sept 2006 - Jun 2010\nSelected coursework: Calculus I & II, Linear Algebra, Probability Theory and Mathematical Statistics,\nFunction of Complex Variables, Mathematical Physics Equations, Computation Methods.\nData Science Certificate, Coursera May 2015 - Dec 2015\nSelected coursework: Machine Learning, R language, The Data Scientist's Toolbox, Statistical Inference,\nExploratory Data Analysis, Regression Model.\nProfessional Skills\nProgramming Languages: Proficient: Python Matlab Familiar: SQL R Scala\nWeb Develop

In [162]:
d = {'bachelor': 210, 'master': 76}

In [184]:
for i in d.keys():
    print(i)

bachelor
master


In [301]:
user_record_degree['education'].lower()

'university of wisconsin-madison fall 2013 spring 2016\nmaster of science in statistics & master of science in computer sciences, may 2016\ncoursework\n- statistics: mathematical statistics, regression analysis & anova, financial statistics, biostatistics\n- computer sciences: machine learning, optimization theory (linear & nonlinear), data structure\nsogang university spring 2007 spring 2013\nbachelor of science in chemistry & bachelor of science in economics, august 2013\nhighest ranked student in the graduating class of chemistry department'

In [10]:
tdf = pd.read_csv('Validations.csv')

In [28]:
tdf.head()

,id,S3 key,Institution,Degree Type,Degree Level,Years low,Year high
0,https://s3.amazonaws.com/careers.dataskeptic.c...,resumes/unsourced/text/resumes_unsourced_Young...,"The Ohio State University,Columbus",Computer Science & Engineering,Ph.D,2013,Present
1,https://s3.amazonaws.com/careers.dataskeptic.c...,resumes/unsourced/text/resumes_unsourced_Young...,"Georgia Institute of Technology, Atlanta",Computer Science,MS,2009,2011
2,https://s3.amazonaws.com/careers.dataskeptic.c...,resumes/unsourced/text/resumes_unsourced_Young...,"Handong Global University, Pohang, Korea",Computer Science & Electronic Engineering,BS,2001,2009
3,https://s3.amazonaws.com/careers.dataskeptic.c...,resumes/unsourced/text/resumes_unsourced_Yosso...,University of Rochester,Physics,Ph.D,2008,2014
4,https://s3.amazonaws.com/careers.dataskeptic.c...,resumes/unsourced/text/resumes_unsourced_Yosso...,"The University of California, San Diego",Physics,BS,2003,2007


In [12]:
url = 'https://raw.githubusercontent.com/endSly/world-universities-csv/master/world-universities.csv'
import pandas as pd
data = pd.read_csv(url)
data.columns = ['country','school','web']
data = data[['school']]
data['school'] = [i.lower() for i in data['school']]

In [15]:
user_record_degree=es.search(index=index_name1,query={"query" : {"match":{"_id": 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_christian.muise@ibm.com_CV.pdf.txt'}}})['hits']['hits'][0]['_source']

In [16]:
def get_school_name(user_record_degree):
    content1 = user_record_degree['education'].lower()
    content2 = re.sub('&', 'and', content1)
    content3 = remove_spe_chara(content2)
    matches = []
    for school in data['school']:
        match = content3.find(school)
        leng = len(school)
        matches.append(match)
    indexes = [n for n,i in enumerate(matches) if i>-1]
    univ_pos = [i for i in matches if i != -1]
    schools = []
    length = []
    for idx in indexes:
        leng = int(len(data.iloc[idx]['school']))
        length.append(leng)
    for i , pos in zip(range(len(length)),univ_pos):
        school_name = content3[pos:pos+length[i]]
        schools.append(school_name)
    return schools 

In [18]:
#a = data.loc[data['school'].apply(lambda x: x.find('wisconsin') >=0)]

In [19]:
def remove_spe_chara(string):
    return re.sub(r'\W+', ' ', string)

In [20]:
#data.loc[data['school'].apply(lambda x: x.find('university of science and technology') >=0)]

In [21]:
data['school']=[re.sub('&', 'and', i) for i in data['school']]

In [22]:
data['school'] = data['school'].apply(lambda x: remove_spe_chara(x).lower())

In [23]:
data.drop_duplicates(inplace = True)

In [24]:
data.head()

,school
0,abu dhabi university
1,ajman university of science and technology
2,alain university of science and technology
3,al ghurair university
4,alhosn university


In [25]:
user_record_degree

{'awards': ', Grants, and Honours\nWinner of the International Competition on Knowledge Engineering for Planning and\nScheduling (ICKEPS) . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2016\nBest System Demo at the International Conference on Automated Planning and Scheduling 2016\nNational Sciences and Engineering Research Council (NSERC) Postdoctoral Fellowship . 2016-17\nSelected Participant for the Heidelberg Laureate Forum . . . . . . . . . . . . . . . . . . . . . 2015\nBest dissertation award at the International Conference on Automated Planning and\nScheduling . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2015\nFirst place in the Time Track, and runner up in the Quality and Coverage tracks at the\nCompetition of Distributed and Multiagent Planners . . . . . . . . . . . . . . . . . . . . . . 2015\nGoogle Places API Developer Challenge Judge\'s Choice Award . . . . . . . . . . . . . . . . . 2013\nTop su